In [217]:
import re
import requests
from bs4 import BeautifulSoup
import time
from urllib import parse
import json
from selenium import webdriver

import datetime

import ssl
import urllib.request as req
    



# 영화 제목 
# 일시 
# 장소 


#     ##### 동대문구 크롤러##### (월별)
def dongdaemungu_movie_crawler(libGroup):
    
    params = {
        'libGroup': libGroup,
    }
    url = "http://www.l4d.or.kr/library/index.php?g_page=culture&m_page=culture04&" + parse.urlencode(params)

    
    request = requests.get(url)
    response = request.text
    soup = BeautifulSoup(response, 'lxml')
    
    # 영화별 정보 담은 박스 
    data_boxes = soup.select('div.data_wrapper')

    for data_box in data_boxes:
        
        # 사진 뽑기
        pic_url = data_box.select_one('div.pic > a > img').get('src')
        # 사진 url을 뽑아보니 두가지 경우가 있다. 
        # http로 시작 하는 경우, 상대결로로서 l4d주소 있다 가정하고 뒷주소만 있는경우
        
        # 뒷 주소만 있는경우 ../경로표시 빼고 l4d 주소 더해준다. 
        if not re.findall('http://',pic_url):
            pic_url =  'http://www.l4d.or.kr/'+ re.findall('\..(.*)',pic_url)[0]
        
        #@ 사진
        print(f'pic_url: {pic_url}')
        
        #제목 뽑기 
        title = data_box.select_one('h3.recom_title > a').get_text(strip=True)
        #@ 제목 
        print(f'title: {title}')
        
        # 나머지 정보들 뽑기.
        
#         print(data_box.prettify())
        
#         <li>
#           <span class="fb">
#            일시
#           </span>
#           : 2018.09.02 13:00 (일)
#          </li>
#
#          이런식으로 리스트들이 들어있는데 
#          li를 자식인 span.fb의 text인 일시 를 기준으로 찾고 싶다. 
#          즉 기준이되는 찾은 테그의 부모의 택스트를 가져오고 싶은것.
        
        # 여기서 선택 된 것은 tag리스트이고 
        # 이중 text가 일시 인것을 먼저 찾아보겠다. 
        

        span_tags = data_box.select('ul > li > span.fb')
#         print(span_tags)

        when = ""#일시
        place = ""#장소
        runtime = ""#시간        # 만약에 해당 항목 없으면 ""
        
        for span_tag in span_tags:
            span_tag_text = span_tag.get_text(strip=True)
            
            if span_tag_text == "일시":
                when = span_tag.parent.get_text(strip=True)
                #@ 일시
                print(when)
            
            elif span_tag_text =="장소":
                place = span_tag.parent.get_text(strip=True)
                #@ 장소 
                print(place)
                
            elif span_tag_text =="시간":
                runtime = span_tag.parent.get_text(strip=True)
                #@ 시간
                print(runtime)
                
                
        print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')

        
        
#    ##### 성동구 크롤러 ##### (페이지별)

def seongdonggu_movie_crawler(area_code,year):

    
    # 페이지 1~5를 탐색해서 해당 월에 상영하는 작품만 가져오자.
    for page in range(1,6):
        
        params = {
            'page':page
        }

        url = "https://www.sdlib.or.kr/"+ area_code + "/U07080.asp?viewtype=list&" + parse.urlencode(params)
#     url = "https://www.sdlib.or.kr/SD/U07080.asp?viewtype=list&page=1&libno="

        ssl._create_default_https_context = ssl._create_unverified_context
        res = req.urlopen(url)
        soup = BeautifulSoup(res, 'lxml')

        table_rows = soup.select('table.table700 > tbody > tr')


        for row in table_rows:
            for index, td in enumerate(row.select('td')):
                if index == 0:
                    print(f'time_date : {td.get_text(strip=True)}') #날짜 
                elif index ==2:
                    print(f'time_hour : {td.get_text(strip=True)}') # 시작시간 
                elif index ==3:
                    print(f'title : {td.get_text(strip=True)}') # 영화 제목
                elif index ==6:
                    print(f'runtime :  {td.get_text(strip=True)}') # 상영시간 

            print('@@@@@@@@@@@@@@@@@@@@@@@@@')
            
        print(f'{page}@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
            




    
def gwangjingu_movie_crawler(area_code,year):
    
    def get_soup(url):
        ssl._create_default_https_context = ssl._create_unverified_context
        res = req.urlopen(url)
        soup = BeautifulSoup(res, 'lxml')    
        return soup
        
    if area_code == 'gjinfo':
        url = "https://www.gwangjinlib.seoul.kr/gjinfo/menu/10087/program/30020/movieList.do"
        soup = get_soup(url)
        
        movies = soup.select('ul.movie-list > li')
        for movie in movies:
            photo_url = "https://www.gwangjinlib.seoul.kr" + movie.select_one('div.thumb > img').get('src')
#             print(photo_url) # 사진 
            title =  movie.select_one('dl > dt')
            print(title.contents[1].strip()) # 제목
            dds = movie.select('dl > dd')
            for index, dd in enumerate(dds):
                if index ==1:
                    print(f'런타임: {dd.contents[1].strip()}') # 런타임
                
                elif index ==3:
#                     print(f'상영일자 : {dd.contents[1]}') #상영일자(날짜 + 시간 ) # 2018.10.27(토) 오후 2시 or 오후2시, 오전1시 
                    when = dd.contents[1]
                    when_date = re.findall("(\S*)\(",when)[0]
                    print(f'상영 날짜 : {when_date}') # 상영 날짜  # 2018.10.27

                    when_time_pre = re.findall("\)\s*(.{4,6})시",when)[0]
#                     print(f'상영 시간 : {when_time_pre}') # 상영 시간  # 오후 2시
                    if '오후' in when_time_pre:
                        when_time = int(re.findall("오후(.*\d*)",when_time_pre)[0].strip()) +12

                    elif '오전' in when_time_pre:
                        when_time = int(re.findall("오전(.*\d*)",when_time_pre)[0].strip())
                    
                    print(f'상영 시간 :{when_time}')
                        
                    print('@@@@@@@@@@@@@@@@@@')
                    
                    
            
            
        
    
    elif area_code == 'jgsports':
        url = "https://www.gwangjinlib.seoul.kr/jgsports/menu/11005/program/30201/eventList.do"
        soup = get_soup(url)
        
    elif area_code == 'gu3dong':
        url = "https://www.gwangjinlib.seoul.kr/gu3dong/menu/11002/program/30202/eventList.do"
        soup = get_soup(url)

    table_rows = soup.select('table.table700 > tbody > tr')



def main_movie_crawler():
    
    # 오늘 날짜 먼저 가져옴 
    now = datetime.datetime.now()
    year = now.year
    day = now.day
    month = now.month

    
#     ##### 동대문구 크롤러##### 

#     dongdaemungu_area_code_list = ['MA','MF','MB','MC','MJ'] 
#     # 정보화,답십리, 장안, 용두, 휘셩 (이문 제외)
# #     dongdaemungu_area_code_list = ['MA']
    
#     for libGroup in dongdaemungu_area_code_list:
#         dongdaemungu_movie_crawler(libGroup)
#         print(f'{libGroup} #####################################################################################################################################')



#     ##### 성동구 크롤러 #####
    
#     seongdonggu_area_code_list = ['SD','YD','SS','KH','CG'] 
#     # 성동구립,용답, 장안, 성수, 금호,청계
# #     seongdonggu_area_code_list = ['SD']
    
#     for area_code in seongdonggu_area_code_list:
#         seongdonggu_movie_crawler(area_code,year)
#         print(f'{area_code}#############################################################################################################################')




    ##### 광진구 크롤러 #####
    gwangjingu_area_code_list = ['gjinfo','jgsports','gu3dong']
#     정보 , 중곡문화체육센터, 구의제3동 
    for area_code in gwangjingu_area_code_list:
        gwangjingu_movie_crawler(area_code,year)
        print(f'{area_code}#############################################################################################################################')



    

In [218]:
main_movie_crawler()

행복의 나라
런타임: 86분
상영 날짜 : 2018.10.06
상영 시간 :11
@@@@@@@@@@@@@@@@@@
암살
런타임: 139분
상영 날짜 : 2018.10.06
상영 시간 :14
@@@@@@@@@@@@@@@@@@
토마스와 친구들:용감한 기관차와 괴물소동
런타임: 60분
상영 날짜 : 2018.10.07
상영 시간 :11
@@@@@@@@@@@@@@@@@@
수성못
런타임: 88분
상영 날짜 : 2018.10.13
상영 시간 :11
@@@@@@@@@@@@@@@@@@
위험한 상견례2
런타임: 119분
상영 날짜 : 2018.10.13
상영 시간 :14
@@@@@@@@@@@@@@@@@@
오즈의 마법사: 돌아온 도로시
런타임: 92분
상영 날짜 : 2018.10.14
상영 시간 :11
@@@@@@@@@@@@@@@@@@
파파좀비
런타임: 89분
상영 날짜 : 2018.10.20
상영 시간 :11
@@@@@@@@@@@@@@@@@@
스톰 인사이드
런타임: 83분
상영 날짜 : 2018.10.20
상영 시간 :11
@@@@@@@@@@@@@@@@@@
꼬마잠수함 올리2
런타임: 72분
상영 날짜 : 2018.10.21
상영 시간 :11
@@@@@@@@@@@@@@@@@@
여자들
런타임: 101분
상영 날짜 : 2018.10.27
상영 시간 :11
@@@@@@@@@@@@@@@@@@
미션 임파서블: 로그네이션
런타임: 131분
상영 날짜 : 2018.10.27
상영 시간 :14
@@@@@@@@@@@@@@@@@@
천년여우 여우비
런타임: 85분
상영 날짜 : 2018.10.28
상영 시간 :11
@@@@@@@@@@@@@@@@@@
우리 선생님을 고발합니다.
런타임: 99분
상영 날짜 : 2018.10.31
상영 시간 :14
@@@@@@@@@@@@@@@@@@
gjinfo########################################################################################################################